This scipt is used to get the deployment information of the QRadar console. It will return the following information:
* QRadar version
* QRadar build number
* Hosts types
* Appliances type

This is an example of how to use the IBM QRadar API. Educational purposes only. Please test it on your own risk. Do not test it in production environment.


Import the necessary modules

In [1]:
import json
import requests
import pandas as pd
import urllib.parse
import time
import numpy as np
# from pandas.json_normalize import json_normalize
# this line depends on the version of pandas you have installed
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings("ignore")

You need to change the values of the variables below for your environment

In [2]:
#authentication
SEC_TOKEN = 'xxxxxx-xxxxxx-xxxxxx-xxxxx-xxx'
#token name in Qradar is "Inyector"
URL_base = 'https://xxx.xxx.xxx.xxx/api'
URL_suffix='/config/deployment/hosts'
URL_Concat = URL_base + URL_suffix

In order to get the data, we need to create a header with the security token

In [3]:
#headers
header = {
    'SEC':SEC_TOKEN,
    'Content-Type':'application/json',
    'accept':'application/json'
}

In [4]:
#get all the hosts
response = json.loads(requests.get(URL_Concat, headers=header, verify=False).text)
df=pd.json_normalize(response)
df.columns

Index(['eps_rate_hardware_limit', 'components', 'average_fpm', 'public_ip',
       'secondary_server_id', 'average_eps', 'fpm_allocation',
       'email_server_id', 'private_ip', 'hostname', 'total_memory',
       'primary_server_id', 'remote_tunnel_initiation_enabled',
       'eps_allocation', 'id', 'cpus', 'app_memory', 'peak_eps',
       'proxy_server_id', 'license_serial_number', 'peak_fpm', 'version',
       'fpm_rate_hardware_limit', 'encryption_enabled', 'compression_enabled',
       'status', 'appliance.id', 'appliance.type'],
      dtype='object')

In [5]:
df=df.drop(['average_fpm',
        'secondary_server_id',
        'email_server_id',
        'remote_tunnel_initiation_enabled',
        'proxy_server_id',
        'license_serial_number',
        'peak_fpm',
        'fpm_rate_hardware_limit',
        'fpm_allocation',
        'primary_server_id',
    ], axis=1)

In [6]:
#filer status active if you want to see  only active hosts
#df=df[df['status']=='Active']

In [7]:
#create a new column with the rol
df['rol']=np.nan

In [8]:
#if 'eps_rate_hardware_limit' is null then it is a apphost
df.loc[df['eps_rate_hardware_limit'].isnull(),'rol']='Apphost'

In [9]:
#components as string
df['components']=df['components'].astype(str)
#if 'components' contains 'ariel_proxy_server' then it is a Console
df.loc[df['components'].str.contains('ariel_proxy_server'),'rol']='Console' 
#if 'components' contains 'eventprocessor' then it is a Event Processor
df.loc[df['components'].str.contains('eventprocessor'),'rol']='Event Processor' 
#if 'components' contains 'eventcollector' then it is a Event Collector
df.loc[df['components'].str.contains('eventcollector'),'rol']='Event Collector'

In [10]:
#total_memory to GB and two decimals
df['total_memory']=df['total_memory'].apply(lambda x: round(x/1024/1024,2))
df['app_memory']=df['app_memory'].apply(lambda x: round(x/1024/1024,2))

In [11]:
#if appliance.type is software'
for index, row in df.iterrows():
    if row['appliance.id']=='software':
        df.loc[index,'appliance.type']='Virtual Appliance'
    elif row['appliance.id']=='4000':
        df.loc[index,'appliance.type']='Virtual Appliance'
    else:
        df.loc[index,'appliance.type']='Physical Appliance'

In [18]:
df.head(5)

,eps_rate_hardware_limit,components,public_ip,average_eps,private_ip,hostname,total_memory,eps_allocation,id,cpus,app_memory,peak_eps,version,encryption_enabled,compression_enabled,status,appliance.id,appliance.type,rol
0,100000.0,None,2.2.2.2,None,1.1.1.1,Host example,0.00,NaN,186,0,0.0,None,0.0.0,None,None,ADDING,software,Virtual Appliance,NaN
1,100000.0,"['hostcontext', 'eventcollectoringress', 'even...",2.2.2.2,None,1.1.1.1,Host example,31.26,1500.0,181,16,0.0,None,7.4.3,True,False,Active,software,Virtual Appliance,Event Processor
2,100000.0,"['hostcontext', 'eventcollectoringress', 'even...",2.2.2.2,None,1.1.1.1,Host example,15.51,500.0,182,8,0.0,None,7.4.3,True,False,Active,software,Virtual Appliance,Event Processor
3,100000.0,"['hostcontext', 'eventcollectoringress', 'even...",2.2.2.2,None,1.1.1.1,Host example,29.29,800.0,155,8,0.0,None,7.4.3,False,False,Active,software,Virtual Appliance,Event Processor
4,100000.0,"['hostcontext', 'eventcollectoringress', 'even...",2.2.2.2,None,1.1.1.1,Host example,62.66,5000.0,177,20,0.0,None,7.4.3,False,False,Active,software,Virtual Appliance,Event Processor
